## **1. Research Questions introduction**



	“Quais são os fatores (género, número de ratings, e popularidade) que mais influenciam a média de avaliação dos filmes no MovieLens?”

Isto permite:
	•	Explorar a estrutura e a distribuição dos dados (EDA);
	•	Criar métricas derivadas (popularidade, número de votos, média ponderada);
	•	Usar modelos simples (regressão linear / árvore de decisão) para quantificar o impacto de cada variável;
	•	E escalar a análise — localmente em DuckDB, e facilmente replicável em cloud (Athena/BigQuery).

## **2. Pipeline**

### **2.1. Import necessary libraries and packages**

In [1]:
import sys
print(sys.executable)

#"C:\Users\SaraEstevesHenriques\AppData\Local\Programs\Python\Python313\python.exe" -m pip install polars

c:\Users\SaraEstevesHenriques\AppData\Local\Programs\Python\Python313\python.exe


In [2]:
import polars as pl
import sys
import os

In [4]:
print ("This is the file directory:", os.getcwd())
parquet_path = os.path.join(os.getcwd(), "moviedetails.parquet")

print("Python executable:", sys.executable)
print("Parquet exists:", os.path.exists(parquet_path))


This is the file directory: c:\Users\SaraEstevesHenriques\Documents\GitHub\BDF25_7\.git\BDF25_7\BDF25_7_data\big_data\ml-32m
Python executable: c:\Users\SaraEstevesHenriques\AppData\Local\Programs\Python\Python313\python.exe
Parquet exists: True


In [6]:
# Eager read (loads into memory)
try:
    df = pl.read_parquet(parquet_path)
    print("Eager read shape:", df.shape)
    display(df.head())
except Exception as e:
    print("Eager read failed:", e)

Eager read shape: (9255, 6)


movieId,title,genres,average_rating,rating_count,tag_count_per_movie
i64,str,str,f64,i64,i64
161582,"""Hell or High Water (2016)""","""Crime|Drama""",3.5625,3176,3176
1982,"""Halloween (1978)""","""Horror""",3.722222,8586,8586
52245,"""Blades of Glory (2007)""","""Comedy|Romance""",3.088235,2482,2482
62,"""Mr. Holland's Opus (1995)""","""Drama""",3.70625,6800,6800
508,"""Philadelphia (1993)""","""Drama""",3.613636,27324,27324


In [ ]:
# Lazy read (better for very large datasets)
try:
    lf = pl.scan_parquet(parquet_path)
    print("LazyFrame created. Example - first 5 rows:")
    print(lf.limit(5).collect())
except Exception as e:
    print("Lazy read failed:", e)

### **2.1. Data Preparation**

2.	Calcular:

•nº total de utilizadores, filmes e avaliações - não sei se o número de users é preciso
•distribuição de rating (média, mediana, desvio padrão)
•nº de filmes por género
•nº médio de ratings por filme e por utilizador

3.	Visualizar:
	
•histograma das classificações
•top 10 géneros mais avaliados
•relação entre nº de ratings e média por filme


#### 2.1.1. Titles analysis

The titles in most cases have both the actual title and the movie year. The first step is to split these informations. 

In [19]:
df.head(20)

movieId,title,genres,average_rating,rating_count,tag_count_per_movie
i64,str,str,f64,i64,i64
161582,"""Hell or High Water (2016)""","""Crime|Drama""",3.5625,3176,3176
1982,"""Halloween (1978)""","""Horror""",3.722222,8586,8586
52245,"""Blades of Glory (2007)""","""Comedy|Romance""",3.088235,2482,2482
62,"""Mr. Holland's Opus (1995)""","""Drama""",3.70625,6800,6800
508,"""Philadelphia (1993)""","""Drama""",3.613636,27324,27324
…,…,…,…,…,…
1210,"""Star Wars: Episode VI - Return…","""Action|Adventure|Sci-Fi""",4.137755,281260,281260
1732,"""Big Lebowski, The (1998)""","""Comedy|Crime""",3.924528,235108,235108
1956,"""Ordinary People (1980)""","""Drama""",4.0,1500,1500


In [20]:
# Extract year and remove it from title
df = df.with_columns([
    # Extract year, the 4 numbers inside the "". If no year is found it will be null
    pl.col("title").str.extract(r'\((\d{4})\)', 1).cast(pl.Int64).alias("year"),
    # Remove year from title (removes the pattern " (YYYY)" or "(YYYY)")
    pl.col("title").str.replace(r'\s*\(\d{4}\)', '').str.strip_chars('"').alias("title")
])

In [21]:
df.head()

movieId,title,genres,average_rating,rating_count,tag_count_per_movie,year
i64,str,str,f64,i64,i64,i64
161582,"""Hell or High Water""","""Crime|Drama""",3.5625,3176,3176,2016
1982,"""Halloween""","""Horror""",3.722222,8586,8586,1978
52245,"""Blades of Glory""","""Comedy|Romance""",3.088235,2482,2482,2007
62,"""Mr. Holland's Opus""","""Drama""",3.70625,6800,6800,1995
508,"""Philadelphia""","""Drama""",3.613636,27324,27324,1993


The following line allows to conclude that not every movie has the correspondent year set.

In [26]:
df.null_count()
#12 missing values on the year

movieId,title,genres,average_rating,rating_count,tag_count_per_movie,year
u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,12


In [32]:
df_noyear = df.filter(pl.col("year").is_null())
df_noyear.head (12)

movieId,title,genres,average_rating,rating_count,tag_count_per_movie,year
i64,str,str,f64,i64,i64,i64
171631,"""Maria Bamford: Old Baby""",null,1.0,1,1,null
162414,"""Moonlight""","""Drama""",5.0,537,537,null
171495,"""Cosmos""",null,4.5,334,334,null
176601,"""Black Mirror""",null,5.0,27,27,null
156605,"""Paterson""",null,4.5,155,155,null
…,…,…,…,…,…,…
143410,"""Hyena Road""",null,2.0,7,7,null
149334,"""Nocturnal Animals""","""Drama|Thriller""",3.0,365,365,null
140956,"""Ready Player One""","""Action|Sci-Fi|Thriller""",3.5,5648,5648,null


#### 2.1.2. Genres

There are movies with multiple genres and others with no gender indicated "(no genres listed)". One possible approach is to use explode funcion, creating a line per movie.

In [30]:
#Tranform no (no genres list) to null values
df = df.with_columns(
    pl.when(pl.col("genres").str.to_lowercase().str.strip_chars() == "(no genres listed)")
    .then(None)
    .otherwise(pl.col("genres"))
    .alias("genres")
    )

In [ ]:
df_nogenre = df.filter(pl.col("genres").is_null())


movieId,title,genres,average_rating,rating_count,tag_count_per_movie,year
u32,u32,u32,u32,u32,u32,u32
0,0,30,0,0,0,12


In [37]:

df.null_count()


movieId,title,genres,average_rating,rating_count,tag_count_per_movie,year
u32,u32,u32,u32,u32,u32,u32
0,0,30,0,0,0,12


In [ ]:
df_exploded = df.with_columns(
    pl.col("genres").str.split("|")
).explode("genres")

df_exploded.head()

### **2.2. Exploratory data analysis**

### **2.3. Feature Engineering**

Criar novas variáveis:
-	n_ratings → nº de avaliações por filme
-	avg_rating → média de rating por filme
-	genre_count → nº de géneros atribuídos
-	popularity_score = log(1 + nº de ratings) × média (para normalizar popularidade)

## **3. Models**

Objetivo: medir o peso de cada fator sobre a média de avaliação.
Modelos possíveis:
	•	Regressão Linear (OLS) → prever avg_rating com base em n_ratings, genre_count e dummies dos géneros principais.
	•	Árvore de decisão / Random Forest → avaliar a importância relativa das variáveis.
Métricas: R², RMSE, importância de features.


## **4. Visualization and Anaysis**

Gráficos:
	•	Scatter n_ratings vs avg_rating (com tendência)
	•	Boxplots de avg_rating por género
	•	Importância das features no modelo
	•	Insights:
	•	Géneros com maiores médias
	•	Géneros mais populares
	•	Relação entre popularidade e qualidade percebida


## **5. Escalability and big_data**

Executar as queries principais em DuckDB e exportar para Parquet.
	•	Demonstrar (mesmo que parcialmente) a execução de consultas analíticas escaláveis:

SELECT genre, COUNT(*) AS n_filmes, AVG(rating) AS avg_rating
FROM ratings
JOIN movies USING(movieId)
GROUP BY genre;

	•	Mostrar que a mesma lógica pode ser usada em Athena / BigQuery com datasets maiores (ex: MovieLens 25M).

## **5. Conclusions**

	•	Identificar que fatores explicam melhor as boas avaliações.
	•	Propor extensão:
	•	Modelo preditivo por utilizador (colaborativo)
	•	Integração com tags para enriquecer o conteúdo
